In [ ]:
import pandas as pd
import os
from neurojit.commit import Mining
from environment import (
    PROJECTS
)

# Filter commits that no ast change and concat cuf and baseline data into one file in `../data/dataset/filtered/`

for project in PROJECTS:
    baseline_data = pd.read_csv(f"../data/dataset/baseline/{project}.csv", index_col=0)
    cuf = pd.read_csv(f"../data/dataset/cuf/{project}.csv", index_col=0)
    assert(set(baseline_data.index) == set(cuf.index))
    data = pd.concat([baseline_data, cuf.drop(labels=['buggy', 'repo', 'target', 'project', 'date', 'gap', 'Unnamed: 0'], axis=1)], axis=1)
    # print(data.columns)
    
    for commit_hash in data.index:
        commit = Mining.load("../data/cache", project, commit_hash)
        if commit is None:
            print(f"Commit {commit_hash} not found")
            data.drop(index=commit_hash, inplace=True)
            continue
        methods_len = len(commit.methods_after)
        asts = {}
        will_remove = set()
        for method in commit.methods_after:
            representation = ""
            for path, node in method.ast:
                representation += node.__repr__()
            asts[method.signature] = representation


        for method in commit.methods_before:
            representation = ""
            for path, node in method.ast:
                representation += node.__repr__()
            
            if asts[method.signature] == representation:
                # print(f"https://github.com/apache/{project}/commit/{commit_hash}")
                will_remove.add(method)

        for method in will_remove:
            commit.methods_after.remove(method)
            commit.methods_before.remove(method)

        if len(commit.methods_after) == 0:
            print(f"Empty commit {commit_hash}")
            # Remove commit from cache
            os.remove(f"../data/cache/{project}/{commit_hash}.pkl")
            # Remove commit from data
            data.drop(index=commit_hash, inplace=True)
        elif methods_len != len(commit.methods_after):
            # print(f"Removed {methods_len - len(commit.methods_after)} methods from {commit_hash}")
            Mining.save(commit, "../data/cache")

    data.to_csv(f"../data/dataset/filtered/{project}.csv")
    print(f"Filtered {project}")